# VESC Dashboard - Interactive Notebook Version

This notebook provides a real-time dashboard for monitoring VESC motor controllers. Unlike the command-line version, this updates in place without creating endless scrolling output.

## Features
- **Real-time data display** that updates in the same location
- **Connection status** monitoring
- **Temperature warnings** with color coding
- **System statistics** and message rates
- **Clean, organized layout** optimized for Jupyter notebooks

💡 **Tip**: Try manually spinning the motor while the dashboard is running to see live data changes!

## Setup - Connect to VESC

In [ ]:
# Setup Python path to find student_api from parent directory
import sys
import os
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))))

# Import required libraries
from student_api import VESCStudentAPI
import time
from datetime import datetime
from IPython.display import display, clear_output, HTML
import ipywidgets as widgets
from threading import Thread
import warnings
warnings.filterwarnings('ignore')

# Create API instance
vesc_api = VESCStudentAPI()

# Start the VESC system
if vesc_api.start():
    print("VESC system started successfully\!")
    
    # Get controller for VESC ID 74 (as specified in README)
    vesc = vesc_api.get_controller(74)
    
    if vesc:
        print("Connected to VESC controller\!")
    else:
        print("Failed to get VESC controller")
else:
    print("Failed to start VESC system")
    vesc = None

## Dashboard Helper Functions

In [ ]:
def format_value(value, unit="", decimals=2):
    """Format a value with proper handling of None"""
    if value is None:
        return "---"
    
    if isinstance(value, (int, float)):
        if decimals == 0:
            return f"{int(value)}{unit}"
        else:
            return f"{value:.{decimals}f}{unit}"
    return str(value) + unit

def get_temp_status(temp, warning_thresh, critical_thresh):
    """Get temperature status with color coding"""
    if temp is None:
        return "", "#666666"  # Gray for unknown
    
    if temp >= critical_thresh:
        return "🔴 CRITICAL", "#ff4444"
    elif temp >= warning_thresh:
        return "🟡 WARNING", "#ffaa00"
    else:
        return "🟢 OK", "#44ff44"

def get_connection_status(controller):
    """Get controller connection status"""
    if controller is None:
        return "🔴 NO CONTROLLER", "#ff4444"
    
    try:
        connected = controller.is_connected()
        if connected:
            return "🟢 CONNECTED", "#44ff44"
        else:
            return "🔴 DISCONNECTED", "#ff4444"
    except:
        return "🟡 UNKNOWN", "#ffaa00"

print("✅ Helper functions loaded!")

## Single Update Dashboard

Run this cell to see a **single snapshot** of the VESC data. Good for checking current status without continuous updates.

In [ ]:
def create_dashboard_html(vesc, vesc_api):
    """Create HTML dashboard content"""
    
    # Get current time
    current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    # Check connection
    status_text, status_color = get_connection_status(vesc)
    
    html = f"""
    <div style="font-family: 'Courier New', monospace; background-color: #f8f9fa; padding: 20px; border-radius: 10px; border: 2px solid #dee2e6;">
        <h2 style="text-align: center; color: #333; margin-bottom: 5px;">🚗 VESC MOTOR CONTROLLER DASHBOARD</h2>
        <p style="text-align: center; color: #666; margin-bottom: 20px;">📅 {current_time} | Controller ID: 74</p>
        
        <div style="text-align: center; margin-bottom: 20px;">
            <span style="background-color: {status_color}; color: white; padding: 5px 15px; border-radius: 20px; font-weight: bold;">
                {status_text}
            </span>
        </div>
    """
    
    if vesc is None or not get_connection_status(vesc)[0].startswith("🟢"):
        html += """
        <div style="text-align: center; color: #ff4444; margin: 20px;">
            <p>⚠️ No data available - controller not responding</p>
            <p>• Check CAN connections</p>
            <p>• Verify VESC is powered on</p>
            <p>• Ensure CAN messages are enabled</p>
        </div>
        </div>
        """
        return html
    
    # Get telemetry data
    try:
        telemetry = vesc.get_all_telemetry()
        motor = telemetry.get('motor', {})
        power = telemetry.get('power', {})
        temps = telemetry.get('temperatures', {})
        sensors = telemetry.get('sensors', {})
        
        # Create sections
        html += """
        <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 20px; margin-bottom: 20px;">
        """
        
        # Motor Status Section
        duty_cycle = motor.get('duty_cycle')
        duty_percent = (duty_cycle * 100) if duty_cycle is not None else None
        
        html += f"""
        <div style="background-color: white; padding: 15px; border-radius: 8px; border-left: 4px solid #007bff;">
            <h3 style="color: #007bff; margin-top: 0;">🔧 MOTOR STATUS</h3>
            <table style="width: 100%; font-size: 14px;">
                <tr><td><strong>RPM:</strong></td><td style="text-align: right;">{format_value(motor.get('rpm'), ' rpm', 0)}</td></tr>
                <tr><td><strong>Current:</strong></td><td style="text-align: right;">{format_value(motor.get('current'), ' A')}</td></tr>
                <tr><td><strong>Duty Cycle:</strong></td><td style="text-align: right;">{format_value(duty_percent, ' %', 1)}</td></tr>
            </table>
        </div>
        """
        
        # Power System Section
        html += f"""
        <div style="background-color: white; padding: 15px; border-radius: 8px; border-left: 4px solid #28a745;">
            <h3 style="color: #28a745; margin-top: 0;">⚡ POWER SYSTEM</h3>
            <table style="width: 100%; font-size: 14px;">
                <tr><td><strong>Input Voltage:</strong></td><td style="text-align: right;">{format_value(power.get('input_voltage'), ' V')}</td></tr>
                <tr><td><strong>Input Current:</strong></td><td style="text-align: right;">{format_value(power.get('input_current'), ' A')}</td></tr>
                <tr><td><strong>Amp Hours Used:</strong></td><td style="text-align: right;">{format_value(power.get('amp_hours_consumed'), ' Ah')}</td></tr>
                <tr><td><strong>Amp Hours Regen:</strong></td><td style="text-align: right;">{format_value(power.get('amp_hours_charged'), ' Ah')}</td></tr>
            </table>
        </div>
        """
        
        html += "</div>"  # Close grid
        
        # Temperature Section
        fet_temp = temps.get('fet')
        motor_temp = temps.get('motor')
        fet_status, fet_color = get_temp_status(fet_temp, 80, 90)
        motor_status, motor_color = get_temp_status(motor_temp, 100, 120)
        
        html += f"""
        <div style="background-color: white; padding: 15px; border-radius: 8px; border-left: 4px solid #dc3545; margin-bottom: 20px;">
            <h3 style="color: #dc3545; margin-top: 0;">🌡️ TEMPERATURES</h3>
            <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 20px;">
                <div>
                    <p><strong>FET Temperature:</strong> {format_value(fet_temp, ' °C')}</p>
                    <span style="background-color: {fet_color}; color: white; padding: 3px 8px; border-radius: 12px; font-size: 12px;">
                        {fet_status}
                    </span>
                </div>
                <div>
                    <p><strong>Motor Temperature:</strong> {format_value(motor_temp, ' °C')}</p>
                    <span style="background-color: {motor_color}; color: white; padding: 3px 8px; border-radius: 12px; font-size: 12px;">
                        {motor_status}
                    </span>
                </div>
            </div>
        </div>
        """
        
        # Sensors Section
        html += f"""
        <div style="background-color: white; padding: 15px; border-radius: 8px; border-left: 4px solid #6f42c1; margin-bottom: 20px;">
            <h3 style="color: #6f42c1; margin-top: 0;">📊 SENSORS</h3>
            <div style="display: grid; grid-template-columns: 1fr 1fr 1fr; gap: 15px; font-size: 14px;">
                <div><strong>Tachometer:</strong><br>{format_value(sensors.get('tachometer'), '', 0)}</div>
                <div><strong>PID Position:</strong><br>{format_value(sensors.get('pid_position'), ' °')}</div>
                <div><strong>ADC EXT:</strong><br>{format_value(sensors.get('adc_ext'), ' V')}</div>
                <div><strong>ADC EXT2:</strong><br>{format_value(sensors.get('adc_ext2'), ' V')}</div>
                <div><strong>ADC EXT3:</strong><br>{format_value(sensors.get('adc_ext3'), ' V')}</div>
                <div><strong>Servo/PPM:</strong><br>{format_value(sensors.get('servo_value'), '')}</div>
            </div>
        </div>
        """
        
        # Data freshness
        timestamp = telemetry.get('timestamp', 0)
        if timestamp > 0:
            age = time.time() - timestamp
            if age < 1.0:
                freshness = "🟢 FRESH"
                freshness_color = "#44ff44"
            elif age < 5.0:
                freshness = "🟡 STALE"
                freshness_color = "#ffaa00"
            else:
                freshness = "🔴 OLD"
                freshness_color = "#ff4444"
            
            html += f"""
            <div style="text-align: center; margin-top: 10px;">
                <span style="background-color: {freshness_color}; color: white; padding: 5px 10px; border-radius: 15px; font-size: 12px;">
                    📡 DATA: {freshness} (age: {age:.2f}s)
                </span>
            </div>
            """
        
    except Exception as e:
        html += f"""
        <div style="text-align: center; color: #ff4444; margin: 20px;">
            <p>❌ Error getting telemetry: {e}</p>
        </div>
        """
    
    html += "</div>"
    return html

# Display single dashboard update
dashboard_html = create_dashboard_html(vesc, vesc_api)
display(HTML(dashboard_html))

print("💡 This is a single snapshot. Run the next cell for continuous updates!")

## Live Dashboard with Continuous Updates

This cell creates a **live dashboard** that updates continuously in place. The display refreshes every 200ms without creating new output.

🎮 **Controls:**
- **Run the cell** to start the live dashboard
- **Interrupt the kernel** (⏹️ button) to stop
- **Try spinning the motor** while it's running to see live changes!

⚠️ **Note**: This will run continuously until you stop it!

In [ ]:
# Live Dashboard - Updates in place
print("🚀 Starting Live Dashboard...")
print("💡 Spin the motor by hand to see live data changes!")
print("🛑 Click the ⏹️ (interrupt) button to stop\n")

try:
    update_count = 0
    while True:
        # Clear previous output and show new dashboard
        clear_output(wait=True)
        
        # Create and display dashboard
        dashboard_html = create_dashboard_html(vesc, vesc_api)
        display(HTML(dashboard_html))
        
        # Show update info
        update_count += 1
        print(f"📊 Live Dashboard (Update #{update_count})")
        print(f"🔄 Refreshing every 200ms | 🛑 Interrupt kernel to stop")
        print(f"💡 Try spinning the motor by hand to see data change!")
        
        # Wait before next update
        time.sleep(0.2)  # 5Hz update rate
        
except KeyboardInterrupt:
    print("\n🛑 Live dashboard stopped by user")
except Exception as e:
    print(f"\n❌ Dashboard error: {e}")
finally:
    print("\n✅ Dashboard stopped")

## Customizable Dashboard

Want to adjust the update rate or focus on specific data? Use this customizable version:

In [ ]:
def run_custom_dashboard(duration_seconds=30, update_rate_hz=5, show_sensors=True):
    """Run dashboard with custom settings"""
    
    print(f"🚀 Starting Custom Dashboard for {duration_seconds} seconds at {update_rate_hz}Hz")
    print(f"📊 Sensors display: {'ON' if show_sensors else 'OFF'}")
    print("💡 Spin the motor by hand to see live data changes!\n")
    
    start_time = time.time()
    update_count = 0
    
    try:
        while time.time() - start_time < duration_seconds:
            # Clear and update display
            clear_output(wait=True)
            
            # Create dashboard HTML
            dashboard_html = create_dashboard_html(vesc, vesc_api)
            
            # Modify HTML to hide sensors if requested
            if not show_sensors:
                # Simple way to hide sensors section
                dashboard_html = dashboard_html.replace('📊 SENSORS', '📊 SENSORS (HIDDEN)')
                dashboard_html = dashboard_html.replace('grid-template-columns: 1fr 1fr 1fr', 'display: none')
            
            display(HTML(dashboard_html))
            
            # Show progress
            update_count += 1
            elapsed = time.time() - start_time
            remaining = duration_seconds - elapsed
            
            print(f"📊 Custom Dashboard (Update #{update_count})")
            print(f"⏱️ Time remaining: {remaining:.1f}s | Updates: {update_rate_hz}Hz")
            print(f"💡 Spin the motor to see data change!")
            
            time.sleep(1.0 / update_rate_hz)
            
    except KeyboardInterrupt:
        print("\n🛑 Custom dashboard stopped by user")
    except Exception as e:
        print(f"\n❌ Dashboard error: {e}")
    
    print("\n✅ Custom dashboard complete!")

# Example: Run for 30 seconds at 2Hz with sensors hidden
run_custom_dashboard(duration_seconds=30, update_rate_hz=2, show_sensors=False)

## System Statistics Dashboard

Want to see what's happening under the hood? This shows system statistics and message rates:

In [ ]:
def create_stats_dashboard():
    """Create system statistics dashboard"""
    
    try:
        interface = vesc_api.system_manager.get_interface()
        stats = interface.get_statistics()
        
        html = f"""
        <div style="font-family: 'Courier New', monospace; background-color: #f8f9fa; padding: 20px; border-radius: 10px; border: 2px solid #dee2e6;">
            <h2 style="text-align: center; color: #333; margin-bottom: 20px;">📈 VESC SYSTEM STATISTICS</h2>
            
            <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 20px;">
                <div style="background-color: white; padding: 15px; border-radius: 8px; border-left: 4px solid #007bff;">
                    <h3 style="color: #007bff; margin-top: 0;">📨 MESSAGE STATISTICS</h3>
                    <table style="width: 100%; font-size: 14px;">
                        <tr><td><strong>Messages Received:</strong></td><td style="text-align: right;">{stats['messages_received']:,}</td></tr>
                        <tr><td><strong>Messages Parsed:</strong></td><td style="text-align: right;">{stats['messages_parsed']:,}</td></tr>
                        <tr><td><strong>Parse Errors:</strong></td><td style="text-align: right;">{stats['parse_errors']:,}</td></tr>
                    </table>
                </div>
                
                <div style="background-color: white; padding: 15px; border-radius: 8px; border-left: 4px solid #28a745;">
                    <h3 style="color: #28a745; margin-top: 0;">🎮 COMMAND STATISTICS</h3>
                    <table style="width: 100%; font-size: 14px;">
                        <tr><td><strong>Commands Sent:</strong></td><td style="text-align: right;">{stats['commands_sent']:,}</td></tr>
                        <tr><td><strong>Commands Successful:</strong></td><td style="text-align: right;">{stats['commands_successful']:,}</td></tr>
                        <tr><td><strong>Commands Timeout:</strong></td><td style="text-align: right;">{stats['commands_timeout']:,}</td></tr>
                    </table>
                </div>
            </div>
            
            <div style="text-align: center; margin-top: 20px; background-color: white; padding: 10px; border-radius: 8px;">
                <p style="margin: 0; color: #666;">📊 Success Rate: {(stats['commands_successful'] / max(stats['commands_sent'], 1) * 100):.1f}% | Parse Rate: {(stats['messages_parsed'] / max(stats['messages_received'], 1) * 100):.1f}%</p>
            </div>
        </div>
        """
        
    except Exception as e:
        html = f"""
        <div style="font-family: 'Courier New', monospace; background-color: #f8f9fa; padding: 20px; border-radius: 10px; border: 2px solid #dee2e6;">
            <h2 style="text-align: center; color: #333;">📈 SYSTEM STATISTICS</h2>
            <p style="text-align: center; color: #ff4444;">❌ Error getting statistics: {e}</p>
        </div>
        """
    
    return html

# Show system statistics
stats_html = create_stats_dashboard()
display(HTML(stats_html))

print("💡 These statistics show the health of your VESC communication system!")

## Dashboard Controls and Tips

### 🎮 **How to Use the Dashboard:**
1. **Single Update**: Run the "Single Update Dashboard" cell for a one-time snapshot
2. **Live Updates**: Run the "Live Dashboard" cell for continuous updates
3. **Custom Dashboard**: Adjust timing and display options
4. **System Stats**: Monitor communication health

### 💡 **Getting Interesting Data:**
- **Spin the motor by hand** while dashboard is running
- **Try different spin speeds** - slow vs fast
- **Spin both directions** to see positive/negative values
- **Watch temperature changes** during extended use

### 🔧 **Troubleshooting:**
- **"DISCONNECTED" status**: Check CAN connections and power
- **All zeros**: Normal when motor isn't moving - try spinning
- **High parse errors**: Check CAN bus quality
- **Stale data**: VESC may not be sending status messages

### 🚀 **Advanced Tips:**
- **Monitor temperatures** during heavy use
- **Watch energy recovery** when spinning down
- **Compare different update rates** for performance
- **Use custom dashboard** to focus on specific measurements

The notebook dashboard provides a much cleaner experience than the command-line version - no more endless scrolling! 📊✨

## Cleanup

Run this when you're done to properly close the connection:

In [ ]:
# Cleanup - stop the VESC system
if 'vesc_api' in locals():
    vesc_api.stop()
    print("✅ VESC system stopped and connections closed")
else:
    print("No VESC system to stop")